In [3]:
import pandas as pd # type: ignore

In [ ]:
df = pd.read_csv("news_1.csv")
df.head()

In [ ]:
df.dropna(inplace=True)
df.shape

In [ ]:
df["TitleContent"] = df['title'] + df["content"]
df.head()

In [23]:
from openai import OpenAI
client = OpenAI(api_key="")

def get_embedding(text, model="text-embedding-ada-002"):
    text = text.replace("\n", " ")
    return client.embeddings.create(input = [text], model=model).data[0].embedding

In [8]:
sample_embedding = get_embedding("magic")

In [ ]:
len(sample_embedding)

In [24]:
df["TitleContentVector"] = df["TitleContent"].apply(lambda x: get_embedding(x, model='text-embedding-ada-002'))
df.to_csv("embedded_1.csv")

In [ ]:
from elasticsearch import Elasticsearch
es = Elasticsearch(
    "https://localhost:9200",
    basic_auth=("elastic", ""),
    ca_certs="/Users/manishyella/Downloads/elasticsearch-8.15.2/config/certs/http_ca.crt"
)
es.ping()

In [27]:
from indexMapping import indexMapping
import numpy as np

In [ ]:
es.indices.create(index="news_content_14", mappings=indexMapping)

In [28]:
embedding_df = pd.read_csv("embedded_1.csv", index_col=0)
embedding_df["TitleContentVector"] = embedding_df.TitleContentVector.apply(eval).apply(np.array)

In [ ]:
docs = embedding_df.to_dict("records")
docs[:6]

In [36]:
for doc in docs:
    try:
        if "id" in doc:
            es.index(index="news_content_14", document=doc, id=doc["id"])
        else:
            print(f"ID missing in document: {doc}")
    except Exception as e:
        print(f"Error indexing document with ID {doc.get('id', 'Unknown')}: {e}")

In [ ]:
es.count(index="news_content_14")

In [40]:
input_keyword = "facebook attributes certain filtering problems on human judges rather than artificial intelligence."
vector_of_input_keyword = get_embedding(input_keyword)

In [ ]:
query = {
    "field" : "TitleContentVector",
    "query_vector" : vector_of_input_keyword,
    "k" : 10,
    "num_candidates" : 500, 
}

res = es.knn_search(index="news_content_14", knn=query , source=["title","author","content","image_src"])
res["hits"]["hits"]